In [6]:

def get_sqlite_table_names(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Query to get all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    conn.close()
    return [table[0] for table in tables]

# Usage
tables = get_sqlite_table_names('chinook.db')
print("Tables in SQLite database:", tables)

Tables in SQLite database: ['albums', 'sqlite_sequence', 'artists', 'customers', 'employees', 'genres', 'invoices', 'invoice_items', 'media_types', 'playlists', 'playlist_track', 'tracks', 'sqlite_stat1']


In [ ]:
#Merging and joining
#1.1 Inner Join on Chinook Databases
import pandas as pd
import sqlite3
import numpy as np

with sqlite3.connect('chinook.db') as conn:
    customers = pd.read_sql("SELECT * FROM customers", conn)
    invoices = pd.read_sql("SELECT * FROM invoices", conn)
cus_inv = pd.merge(customers, invoices, on='CustomerId', how='inner')
cus_inv.groupby("CustomerId")['Total'].sum()


CustomerId
1    39.62
2    37.62
3    39.62
4    39.62
5    40.62
Name: Total, dtype: float64

In [ ]:
#1.2 Outer Join on Movie Data 
movies = pd.read_csv('movie.csv')

df1 = movies[['director_name', 'color']].copy()
df2 = movies[['director_name', 'num_critic_for_reviews']].copy()

left_join = pd.merge(df1, df2, on='director_name', how='left')
print(f"Rows in left join: {len(left_join)}")

outer_join = pd.merge(df1, df2, on='director_name', how='outer')
print(f"Rows in outer join: {len(outer_join)}")


Rows in left join: 30300
30300
Rows in outer join: 30300


In [ ]:
#2.1 Grouped aggregations on Titanic
titanic = pd.read_excel('titanic.xlsx')
titanic_grouped = titanic.groupby('Pclass').agg(
    AverageAge=('Age', 'mean'),
    TotalFare=('Fare', 'sum'),
    PassengerCount=('PassengerId', 'count')
).reset_index()
print(titanic_grouped)


   Pclass  AverageAge   TotalFare  PassengerCount
0       1   38.233441  18177.4125             216
1       2   29.877630   3801.8417             184
2       3   25.140620   6714.6951             491


In [40]:
#2.2 Multi-level grouping on Movie Data
movies.groupby(['director_name', 'color']).agg({
    'num_critic_for_reviews': 'sum',
    'duration': 'mean',
}).reset_index()



,director_name,color,num_critic_for_reviews,duration
0,A. Raven Cruz,Color,3.0,97.0
1,Aaron Hann,Color,29.0,87.0
2,Aaron Schneider,Color,160.0,100.0
3,Aaron Seltzer,Color,99.0,85.0
4,Abel Ferrara,Color,48.0,99.0
...,...,...,...,...
2485,Zoran Lisinac,Color,17.0,108.0
2486,Álex de la Iglesia,Color,71.0,104.0
2487,Émile Gaudreault,Color,67.0,92.0
2488,Éric Tessier,Color,9.0,99.0


In [ ]:
#2.3 Nested grouping on Flights
flights = pd.read_parquet('flights')
numeric = ['ArrDelay', 'DepDelay']
for col in numeric:
    flights[col] = pd.to_numeric(flights[col], errors='coerce')
    
flights_grouped = flights.groupby(['Year', 'Month']).agg(
    total_flights =('FlightDate', 'size'),
    Average_arrival_delay = ('ArrDelay', 'mean'),
    Maximum_departure_delay = ('DepDelay', 'max')
    ).reset_index()
flights_grouped.head()


,Year,Month,total_flights,Average_arrival_delay,Maximum_departure_delay
0,2022,4,297605,8.354616,2366.0
1,2022,7,593791,9.855766,2556.0


In [75]:
#3.1 Apply a custom function on Titanic
def sort_age(age):
    return 'Child' if age < 18 else 'Adult'
titanic['Age_Group']= titanic["Age"].apply(sort_age)
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,Adult
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,Adult
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,Adult
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,Adult


In [ ]:
#3.2 Normalize Employee Salaries
employee = pd.read_csv('employee.csv')
employee.groupby('DEPARTMENT')["BASE_SALARY"].transform(lambda x: (x - x.mean()) / x.std())



0       2.568478
1      -0.962884
2      -0.892524
3       0.214332
4       0.224533
          ...   
1995   -1.001846
1996    0.412156
1997   -1.001846
1998   -0.286254
1999   -0.491163
Name: BASE_SALARY, Length: 2000, dtype: float64

In [95]:
#3.3 Custom function on Movies
def sort_duration(duration):
    if duration < 60: 
        return 'Short'
    elif duration < 120:
        return 'Medium'
    elif duration > 120:
        return 'Long'
    else:
        return 'None'
movies['Duration_Category'] = movies['duration'].apply(sort_duration)
movies.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,Duration_Category
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,Long
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,Long
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,Long
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,Long
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0,None


In [99]:
#4.1 Pipeline on Titanic
def filter_survivors(df):
    return df[df['Survived'] == 1]

def fill_missing_age(df):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    return df

def add_fare_per_age(df):
    df['Fare_Per_Age'] = df['Fare'] / df['Age']
    return df

titanic_pipeline = (titanic
                    .pipe(filter_survivors)
                    .pipe(fill_missing_age)
                    .pipe(add_fare_per_age))

titanic_pipeline[['Survived', 'Age', 'Fare', 'Fare_Per_Age']].head()

C:\Users\hp\AppData\Local\Temp\ipykernel_1764\3447684299.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].fillna(df['Age'].mean())
C:\Users\hp\AppData\Local\Temp\ipykernel_1764\3447684299.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fare_Per_Age'] = df['Fare'] / df['Age']


,Survived,Age,Fare,Fare_Per_Age
1,1,38.0,71.2833,1.875876
2,1,26.0,7.9250,0.304808
3,1,35.0,53.1000,1.517143
8,1,27.0,11.1333,0.412344
9,1,14.0,30.0708,2.147914


In [ ]:
#4.2 Pipeline on Flights
def pipe_flight(df, col_name: str):
    df1 = df[df["DepDelay"] > 30]

    df1['ArrTime'] = pd.to_numeric(df1['ArrTime'], errors='coerce')
    df1[col_name] = df1['DepDelay'] / (df1['ArrTime'] / 60)  # Assuming ArrTime is in minutes
    return df1

flights.pipe(pipe_flight, 'Delay_Per_Hour')

C:\Users\hp\AppData\Local\Temp\ipykernel_1764\1332788568.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['ArrTime'] = pd.to_numeric(df1['ArrTime'], errors='coerce')
C:\Users\hp\AppData\Local\Temp\ipykernel_1764\1332788568.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[col_name] = df1['DepDelay'] / (df1['ArrTime'] / 60)  # Assuming ArrTime is in minutes


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Delay_Per_Hour
11,2022,2,4,26,2,2022-04-26,OH,20397,OH,N572NN,...,None,None,None,None,None,None,None,None,None,1.552511
32,2022,2,4,13,3,2022-04-13,OH,20397,OH,N576NN,...,None,None,None,None,None,None,None,None,None,1.791045
33,2022,2,4,14,4,2022-04-14,OH,20397,OH,N559NN,...,None,None,None,None,None,None,None,None,None,1.314433
37,2022,2,4,18,1,2022-04-18,OH,20397,OH,N559NN,...,None,None,None,None,None,None,None,None,None,2.077922
44,2022,2,4,25,1,2022-04-25,OH,20397,OH,N556NN,...,None,None,None,None,None,None,None,None,None,2.467772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891379,2022,3,7,30,6,2022-07-30,DL,19790,DL,N675DL,...,None,None,None,None,None,None,None,None,None,1.085181
891380,2022,3,7,31,7,2022-07-31,DL,19790,DL,N6708D,...,None,None,None,None,None,None,None,None,None,1.262420
891389,2022,3,7,9,6,2022-07-09,DL,19790,DL,N171DN,...,None,None,None,None,None,None,None,None,None,2.138229
891394,2022,3,7,14,4,2022-07-14,DL,19790,DL,N583NW,...,None,None,None,None,None,None,None,None,None,25.000000
